In [4]:
import sys
import os
sys.path.append("D:/tennis_v2/")

import cv2
import numpy as np
import requests
from utils.utils import prepare_json, load_config
CURRENT_DIR = 'D:/tennis_v2/'

In [ ]:
def interactive_undistort_video(output_path=None, default_strength=0.99, default_tangential_strength=0.0):
    """
    交互式广角畸变矫正算法（处理视频）
    
    参数：
    video_path: 输入视频路径
    output_path: 输出视频保存路径（可选）
    default_strength: 初始矫正强度（默认0.0）
    
    使用说明：
    1. 按 's' 保存结果
    2. 按 ESC 或 'q' 退出
    3. 滑动条调整矫正强度
    """
    
    config = load_config(f"{CURRENT_DIR}/config.yaml")
    # cap url address
    response  = requests.get(config['cap_url'])
    if response.status_code == 200:   
        rtmp_addr = response.json()['data']['videoUrl']
        print(f'success get the camera: {rtmp_addr}')
    else:
        print(f'Error: {response.status_code}')

    # 打开视频文件
    cap = cv2.VideoCapture(rtmp_addr)
    if not cap.isOpened():
        print("无法打开视频文件")
        return
    
    # 获取视频的宽度和高度
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 生成默认相机矩阵（假设图像中心为光心，焦距为图像宽度的0.7倍）
    fx = w * 0.65  # 1.8
    fy = h * 0.7 # 1.8
    cx = w // 2
    cy = h // 2
    camera_matrix = np.array([[fx, 0, cx],
                              [0, fy, cy],
                              [0, 0, 1]], dtype=np.float32)
    
    # 初始化畸变系数
    dist_coeffs = np.zeros((5, 1), dtype=np.float32)
    
    # 创建窗口和滑动条
    cv2.namedWindow("Undistort Video Demo")
    cv2.createTrackbar("Strength", "Undistort Video Demo", 
                      int(default_strength*1000), 2000, 
                      lambda x: None)  # -2.0到+2.0范围
    
    # 创建滑动条：切向畸变
    cv2.createTrackbar("Tangential Strength", "Undistort Video Demo", 
                       int(default_tangential_strength * 1000), 2000, 
                       lambda x: None)  # -2.0到+2.0范围
    
    
    # 转换滑动条值为实际强度值
    def get_strength():
        pos = cv2.getTrackbarPos("Strength", "Undistort Video Demo")
        return (pos - 1000) / 500.0  # 映射到[-2.0, +2.0] # -0.14, -0.05
    
    def get_tangential_strength():
        pos = cv2.getTrackbarPos("Tangential Strength", "Undistort Video Demo")
        return (pos - 1000) / 500.0  # 映射到[-2.0, +2.0]
    
    
    # 视频写入设置（如果指定了输出路径）
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 可以根据需要更改编码格式
        out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        strength = get_strength()
        tangential_strength = get_tangential_strength()
        
        # 设置径向畸变系数（主要调整k1，k2按比例跟随）
        k1 = strength
        k2 = strength * 0.1
        dist_coeffs[0] = k1
        dist_coeffs[1] = k2
        
        # 设置切向畸变系数
        p1 = -tangential_strength * 0.1  # 切向畸变系数 p1
        p2 = tangential_strength * 0.05   # 切向畸变系数 p2
        dist_coeffs[2] = p1
        dist_coeffs[3] = p2

        # 执行畸变矫正
        undistorted = cv2.undistort(frame, camera_matrix, dist_coeffs)
        
        # 显示对比视图
        combined = np.hstack((frame, undistorted))
        cv2.putText(combined, f"Strength: {strength:.2f}, Tangential Strength: {tangential_strength:.2f}", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        cv2.imshow("Undistort Video Demo", combined)
        
        # 处理键盘输入
        key = cv2.waitKey(1)
        if key in [27, ord('q')]:  # ESC/Q退出
            break
        elif key == ord('s'):     # 保存当前帧
            if output_path:
                out.write(undistorted)
                print("保存视频帧")
    
    # 释放资源
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

In [ ]:
interactive_undistort_video()

success get the camera: rtmp://rtmp03open.ys7.com:1935/v3/openlive/BB8121883_1_2?expire=1740721426&id=815217620731326464&t=4599f32a5a624ae94cfe51d1d065dff034236f5c035ac4aa3638b76ca2194523&ev=100
